# Simple network to minimize CRPS

**Steps (to be updated)**
1. Compute CRPS for raw ensemble and try to reproduce Sebastians results; first probably the monthly data, since this is much easier (maybe in data_io notebook)
2. Set up network structure (first here, then probably in separate module)
3. Try feeding some data to the model and sanity-check the output
4. Run the network on the full dataset.